## Transforming the Data to a desired form.

**Importing the necessary libraries**

In [45]:
import pandas as pd
import numpy as np
#Before starting the transformation I separarted the tag descreption from the values in 2 csv files.
inp=pd.read_csv('input.csv')
info=pd.read_csv('info.csv')

In [46]:
#information on different tags
info.dtypes

ItemId          int64
ComputerId      int64
ItemName       object
Comment        object
Unit          float64
C/S/ID          int64
DataType       object
BitPos          int64
Address        object
MAX           float64
MIN           float64
ScaleMax      float64
ScaleMin      float64
Decimal       float64
dtype: object

In [35]:
#the tag values every 5 milliseconds
inp.dtypes

Date                  object
Time                  object
Milli Sec              int64
L_R1SDW_SPDREF       float64
L_R1SDD_SPDREF       float64
L_R1SDL_SPDREF         int64
L_R1SDW_SPDFBK       float64
L_R1SDD_SPDFBK       float64
L_R1SDL_SPDFBK         int64
L_R1_ODG_AUX_B07       int64
RM1_RUNDOWN_ID_HS      int64
L_R1SD_WS_GAP        float64
L_R1SD_DS_GAP        float64
L_R1SD_WS_SPDREF     float64
L_R1SD_DS_SPDREF     float64
L_R1SDLS_SPDREF        int64
L_R1SD_WS_SPDFBK     float64
L_R1SD_DS_SPDFBK     float64
L_R1SDLS_SPDFBK        int64
L_R1SD_WS_CUR          int64
L_R1SD_DS_CUR          int64
L_R1_CNT_PASS0         int64
dtype: object

In [25]:
# 1sec=1000 milliseconds
# 10 sec= 10000 milliseconds
#since we obtain a value in every 5 milliseconds, 2000 rows of data on the same tag will amount to the aggregation of data in every 10 seconds.
# This amounts to approximately 6 values per tag , we cannot aggregate over the excess records as of now as they do not aggregate to 10 full seconds
# hence only 12000 rows of data can be transformed as of now.
#making a list of all the tag names
tags=['L_R1SDW_SPDREF',	'L_R1SDD_SPDREF'	,'L_R1SDL_SPDREF',	'L_R1SDW_SPDFBK',	'L_R1SDD_SPDFBK',	'L_R1SDL_SPDFBK',	'L_R1_ODG_AUX_B07',	'RM1_RUNDOWN_ID_HS',	'L_R1SD_WS_GAP'	,'L_R1SD_DS_GAP'	,'L_R1SD_WS_SPDREF',	'L_R1SD_DS_SPDREF',	'L_R1SDLS_SPDREF',	'L_R1SD_WS_SPDFBK'	,'L_R1SD_DS_SPDFBK',	'L_R1SDLS_SPDFBK',	'L_R1SD_WS_CUR',	'L_R1SD_DS_CUR',	'L_R1_CNT_PASS0'
]


In [87]:
a=np.arange(0,12000,2000)
b=np.arange(2000,14000,2000)
tagname,tagvalue,timestamp=[],[],[]
ind=[]
def transformer(data):
    k=0
    for tag in tags:
        col=data[tag]
        for i in range (6):
            temp=col[a[i]:(b[i]-1)]
            tagname.append(tag)     #appending the tag name for every 10000 milliseconds or 10 seconds
            tagvalue.append(max(temp)) #appending the max value in time period of 10 seconds 
            timestamp.append(data.Date[b[i]-1]+' '+data.Time[b[i]-1]) #here we are appending the time at which we are aggregating our results, i.e. the last entry in a batch of 2000 records.
            ind.append(k) 
        k=k+1
    return tagname,tagvalue,timestamp,ind
        


In [88]:
#executing the transfromation
tag_name,tag_value,timestamp,ind=transformer(inp)

In [44]:
timestamp[0:10]

['06-05-2022 21:58:43',
 '06-05-2022 21:58:53',
 '06-05-2022 21:59:03',
 '06-05-2022 21:59:13',
 '06-05-2022 21:59:23',
 '06-05-2022 21:59:33',
 '06-05-2022 21:58:43',
 '06-05-2022 21:58:53',
 '06-05-2022 21:59:03',
 '06-05-2022 21:59:13']

In [47]:
#now that we have three of the columns we desire lets put other things in place too.
info


,ItemId,ComputerId,ItemName,Comment,Unit,C/S/ID,DataType,BitPos,Address,MAX,MIN,ScaleMax,ScaleMin,Decimal
0,35827,10,L_R1SDW_SPDREF,R1 Screw Down WS Speed Reference (mps),NaN,0,REAL : Float,0,0x16464,10000.0,-10000.0,10000.0,-10000.0,0.0
1,35828,10,L_R1SDD_SPDREF,R1 Screw Down DS Speed Reference (mps),NaN,0,REAL : Float,0,0x16468,10000.0,-10000.0,10000.0,-10000.0,0.0
2,35829,10,L_R1SDL_SPDREF,R1 Screw Down LS Speed Reference (mps),NaN,0,REAL : Float,0,0x1646c,10000.0,-10000.0,10000.0,-10000.0,0.0
3,35830,10,L_R1SDW_SPDFBK,R1 Screw Down WS Speed Feedback (mps),NaN,0,REAL : Float,0,0x16470,10000.0,-10000.0,10000.0,-10000.0,0.0
4,35831,10,L_R1SDD_SPDFBK,R1 Screw Down DS Speed Feedback (mps),NaN,0,REAL : Float,0,0x16474,10000.0,-10000.0,10000.0,-10000.0,0.0
5,35832,10,L_R1SDL_SPDFBK,R1 Screw Down LS Speed Feedback (mps),NaN,0,REAL : Float,0,0x16478,10000.0,-10000.0,10000.0,-10000.0,0.0
6,35974,10,L_R1_ODG_AUX_B07,R1 ODG AUX output B07,NaN,0,BOOL : Bit Data,7,0x1653c,1.0,0.0,1.0,0.0,0.0
7,83528,10,RM1_RUNDOWN_ID_HS,R! RUNDOWN ID,NaN,0,TEXT : Text Data,0,0x26db0,NaN,NaN,NaN,NaN,NaN
8,83530,10,L_R1SD_WS_GAP,R1 SCREW DOWN WORK SIDE GAP (MM),NaN,0,REAL : Float,0,0x164f4,10000.0,-10000.0,10000.0,-10000.0,0.0
9,83531,10,L_R1SD_DS_GAP,R1 SCREW DOWN DRIVE SIDE GAP (MM),NaN,0,REAL : Float,0,0x164f8,10000.0,-10000.0,10000.0,-10000.0,0.0


In [98]:
# let us generate the remaining columns
tag_id=[]
tag_desc=[]
tag_unit=[]
pos=0
for tag in tag_name:
    temp=info[info.ItemName==tag]
    tag_id.append(temp.loc[ind[pos],'ItemId'])
    tag_desc.append(temp.loc[ind[pos],'Comment'])
    str=temp.loc[ind[pos],'Comment']
    try:
        l=str.index('(')
    except:
        l=len(str)-1
    unit=str[l+1:len(str)-1]
    tag_unit.append(unit)
    pos+=1
        



In [69]:
info[info.ItemName=='L_R1SDLS_SPDREF'].



,ItemId,ComputerId,ItemName,Comment,Unit,C/S/ID,DataType,BitPos,Address,MAX,MIN,ScaleMax,ScaleMin,Decimal
12,83534,10,L_R1SDLS_SPDREF,R1 SCREW DOWN LOW SPEED REF (MPS),NaN,0,REAL : Float,0,0x1646c,10000.0,-10000.0,10000.0,-10000.0,0.0


In [77]:
tag_id[0]

0    35827
Name: ItemId, dtype: int64

In [92]:
tag_desc

['R1 Screw Down WS Speed Reference (mps)',
 'R1 Screw Down WS Speed Reference (mps)',
 'R1 Screw Down WS Speed Reference (mps)',
 'R1 Screw Down WS Speed Reference (mps)',
 'R1 Screw Down WS Speed Reference (mps)',
 'R1 Screw Down WS Speed Reference (mps)',
 'R1 Screw Down DS Speed Reference (mps)',
 'R1 Screw Down DS Speed Reference (mps)',
 'R1 Screw Down DS Speed Reference (mps)',
 'R1 Screw Down DS Speed Reference (mps)',
 'R1 Screw Down DS Speed Reference (mps)',
 'R1 Screw Down DS Speed Reference (mps)',
 'R1 Screw Down LS Speed Reference (mps)',
 'R1 Screw Down LS Speed Reference (mps)',
 'R1 Screw Down LS Speed Reference (mps)',
 'R1 Screw Down LS Speed Reference (mps)',
 'R1 Screw Down LS Speed Reference (mps)',
 'R1 Screw Down LS Speed Reference (mps)',
 'R1 Screw Down WS Speed Feedback (mps)',
 'R1 Screw Down WS Speed Feedback (mps)',
 'R1 Screw Down WS Speed Feedback (mps)',
 'R1 Screw Down WS Speed Feedback (mps)',
 'R1 Screw Down WS Speed Feedback (mps)',
 'R1 Screw Down 

In [99]:
tag_unit

['mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 'mps',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'MM',
 'MM',
 'MM',
 'MM',
 'MM',
 'MM',
 'MM',
 'MM',
 'MM',
 'MM',
 'MM',
 'MM',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 'MPS',
 '%',
 '%',
 '%',
 '%',
 '%',
 '%',
 '%',
 '%',
 '%',
 '%',
 '%',
 '%',
 '',
 '',
 '',
 '',
 '',
 '']

In [103]:
# converting to a datframe then to csv
output={'timestamp':timestamp,'tag_id':tag_id,'tag_name':tag_name,'tag_desc':tag_desc,'tag_value':tag_value,'tag_unit':tag_unit}
f_op=pd.DataFrame(output)


In [104]:
f_op[0:20]

,timestamp,tag_id,tag_name,tag_desc,tag_value,tag_unit
0,06-05-2022 21:58:43,35827,L_R1SDW_SPDREF,R1 Screw Down WS Speed Reference (mps),0.000000,mps
1,06-05-2022 21:58:53,35827,L_R1SDW_SPDREF,R1 Screw Down WS Speed Reference (mps),29.866667,mps
2,06-05-2022 21:59:03,35827,L_R1SDW_SPDREF,R1 Screw Down WS Speed Reference (mps),0.000000,mps
3,06-05-2022 21:59:13,35827,L_R1SDW_SPDREF,R1 Screw Down WS Speed Reference (mps),14.952726,mps
4,06-05-2022 21:59:23,35827,L_R1SDW_SPDREF,R1 Screw Down WS Speed Reference (mps),0.000000,mps
5,06-05-2022 21:59:33,35827,L_R1SDW_SPDREF,R1 Screw Down WS Speed Reference (mps),0.000000,mps
6,06-05-2022 21:58:43,35828,L_R1SDD_SPDREF,R1 Screw Down DS Speed Reference (mps),0.000000,mps
7,06-05-2022 21:58:53,35828,L_R1SDD_SPDREF,R1 Screw Down DS Speed Reference (mps),29.866667,mps
8,06-05-2022 21:59:03,35828,L_R1SDD_SPDREF,R1 Screw Down DS Speed Reference (mps),0.000000,mps
9,06-05-2022 21:59:13,35828,L_R1SDD_SPDREF,R1 Screw Down DS Speed Reference (mps),14.952726,mps


In [105]:
f_op.to_csv('Output.csv')